In [1]:
import os
import json

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.util.tools_fabric import ToolsFabric

from react_agent.src.util.llm_proxy import LLM_PROXY

from experiments.util.fact_score.fact_scorer import FactScorer

In [2]:
USE_MCP = True
DEBUG_MODE = True
INCLUDE_WEBSEARCH_TOOL = True

fact_scorer = FactScorer()
path_to_ressources = "./resources/01_facts/"

files = {
    "extensibility": "extensibility_assistance_facts.json",
    "malaysia": "malaysia_support_facts.json",
    "peppol": "peppol_support_facts.json",
}

In [3]:
file_path = os.path.abspath(path_to_ressources + files["extensibility"])


with open(file_path, encoding="utf8") as f:
    json_file = json.load(f)

In [4]:
case = json_file["cases"][1]

In [5]:
triage_agent = Triage()

query_categorization = triage_agent.triage_user_message(
    user_message=case["question"]
)
if DEBUG_MODE:
    print(f"Query categorization: {query_categorization}")

Query categorization: {'user_query': 'For a Greece customer in Public Cloud, what are the steps to enhance the generated XML before sending it to the Tax Authorities using ADT?', 'category': 'Knowledge-QA'}


In [6]:
tools = ToolsFabric.get_tools_for_category(
    use_mcp=USE_MCP,
    configuration=query_categorization["category"],
    include_web_search=INCLUDE_WEBSEARCH_TOOL,
)

In [7]:
if USE_MCP:
    async with MultiServerMCPClient( tools ) as client:
        agent = ReActAgent(tool_list=client.get_tools())
        if DEBUG_MODE:
            print(agent.create_sys_prompt(), "\n")
        agent_answer = await agent.arun_agent_with_input(
            user_message=query_categorization["user_query"], debug=DEBUG_MODE
        )
else:
    agent = ReActAgent(tool_list=tools)
    if DEBUG_MODE:
        print(agent.create_sys_prompt(), "\n")
    agent_answer = agent.run_agent_with_input(user_message=query_categorization["user_query"], debug=DEBUG_MODE)

if not DEBUG_MODE:
    print(agent_answer)

# Role
You are an expert in Electronic Document Processing, with deep domain knowledge in SAP Document and Reporting Compliance, Peppol, UBL, and eInvoicing standards.

# Objective
Use a reason-and-act (ReAct) approach to answer user questions with clear, well-supported reasoning chains, and tool-validated outputs. Final answers must reflect insights derived from specific tool calls.

# Instructions
**You will operate in a strict step-by-step loop. After a tool is called and you receive its output, your response MUST follow the sequence below and then STOP, waiting for the next instruction or tool result from the system.**
1. Observation: Restate the user's request or define the sub-task being addressed. Clearly establish the current focus.
2. Thought: Analyze the problem. Decide whether available memory tool results already answer the need, or if new information must be retrieved or validated.
3. Action Plan: Generate a high-level sequence outlining how you intend to solve the user's 

In [8]:
output = await fact_scorer.get_fact_score(facts=case.get("facts"), knowledge_source=agent_answer, debug=DEBUG_MODE)
print("Fact score: ", output)

Fact: The task involves enhancing an XML file for a Greece customer in Public Cloud.
Is contained: True
Reason: The context explicitly mentions enhancing the XML for a Greece customer in the Public Cloud using ABAP Development Tools.

Fact: The enhancement is done using ABAP Development Tools (ADT).
Is contained: True
Reason: The context explicitly mentions using ABAP Development Tools (ADT) for enhancing the XML for a Greece customer.

Fact: The enhancement spot used is ES_EDOCUMENT_CLOUD.
Is contained: True
Reason: The context explicitly mentions navigating to the enhancement spot `ES_EDOCUMENT_CLOUD` for implementing enhancements to the XML file.

Fact: A new BAdI Enhancement Implementation is created by right-clicking ES_EDOCUMENT_CLOUD.
Is contained: False
Reason: The context does not mention creating a BAdI Enhancement Implementation by right-clicking ES_EDOCUMENT_CLOUD.

Fact: The implementation involves selecting EDOC_ADAPTOR_CLOUD.
Is contained: True
Reason: The context explic

In [9]:
LLM_PROXY.print_usage()

Call Count: 11
Token Usage: {'input_tokens': 12690, 'output_tokens': 1012, 'total_tokens': 13702}
